**Requirements & Dependencies**

> Install below library

In [7]:
!pip install duckdb
!pip install ipykernel
!pip install pandas

5958.57s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


5965.31s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


5971.88s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


  Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl.metadata (61 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 6.1 MB/s eta 0:00:0000:0100:01
Using cached numpy-1.26.4-cp310-cp310-macosx_11_0_arm64.whl (14.0 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)


**Introduction**

Gaps and Island problem is a classic problem of finding missing values AKA gaps or islands in a sequence of values. For cases, like with finding period of inactivity between certail interval by an employee is a gaps problem while if the data reporting is done at a certain interval and find period of activity across multiple periods is an island problem. For eg: in a table with employee name, floor number the employee has worked on and the start and end time when he worked on the floor

<table>
    <tr>
    <b>
        <td>name</td>
        <td>floor_number</td>
        <td>start_time</td>
        <td>end_time</td>
    </b>
    </tr>
    <tr>
        <td>Mark</td>
        <td>1</td>
        <td>31-May-2024 11:56:00</td>
        <td>31-May-2024 15:21:00</td>
    </tr>
    <tr>
        <td>Mark</td>
        <td>1</td>
        <td>31-May-2024 13:26:00</td>
        <td>31-May-2024 16:12:00</td>
    </tr>
    <tr>
        <td>Mark</td>
        <td>1</td>
        <td>31-May-2024 17:01:00</td>
        <td>31-May-2024 17:31:00</td>
    </tr>
    <tr>
        <td>Ashok</td>
        <td>1</td>
        <td>31-May-2024 10:32:00</td>
        <td>31-May-2024 11:28:00</td>
    </tr>
    <tr>
        <td>Ashok</td>
        <td>1</td>
        <td>31-May-2024 12:29:00</td>
        <td>31-May-2024 14:21:00</td>
    </tr>
</table>


In the above example, this is an island problem of finding continuous periods where an employee kept working. As the inputs or 
rows in the above table has a start and end time for each employee recorded at random intervals with some start and end time 
either overlapping i.e, end time of preceeding record for the employee is greater than start time of next record for the same 
employee. Or for some employee, the end time of preceeding record can be same as start time of next record or the end time and 
start time may not overlap at all.

In such cases, if the ask is the find the duration an employee has kept working on recorded floors, simply subtracting the end
time with the start time of each record may not give correct result and we may be counting the time multiple times if there is an
overlap between 2 records. In such scenarios, problem of finding actual duration spent can be easily solved using using the 
island problem approach.

We will initialize the duckdb database for inserting sample data as shown above.

In [10]:
import pandas as pd

employee_work = pd.DataFrame.from_dict(
    {"name":["Mark","Mark","Mark","Ashok","Ashok"],
    "floor_number":[1,1,1,1,1],
    "start_time":["2024-05-31 11:56:00","2024-05-31 13:26:00","2024-05-31 17:01:00","2024-05-31 10:32:00","2024-05-31 12:29:00"],
    "end_time":["2024-05-31 15:21:00","2024-05-31 16:12:00","2024-05-31 17:31:00","2024-05-31 11:28:00","2024-05-31 14:21:00"]
    })

In [12]:
import duckdb
duckdb.sql("select * from employee_work")

┌─────────┬──────────────┬─────────────────────┬─────────────────────┐
│  name   │ floor_number │     start_time      │      end_time       │
│ varchar │    int64     │       varchar       │       varchar       │
├─────────┼──────────────┼─────────────────────┼─────────────────────┤
│ Mark    │            1 │ 2024-05-31 11:56:00 │ 2024-05-31 15:21:00 │
│ Mark    │            1 │ 2024-05-31 13:26:00 │ 2024-05-31 16:12:00 │
│ Mark    │            1 │ 2024-05-31 17:01:00 │ 2024-05-31 17:31:00 │
│ Ashok   │            1 │ 2024-05-31 10:32:00 │ 2024-05-31 11:28:00 │
│ Ashok   │            1 │ 2024-05-31 12:29:00 │ 2024-05-31 14:21:00 │
└─────────┴──────────────┴─────────────────────┴─────────────────────┘

**Solution**

To solve an island problem, we can simply use SQL to get the final solution. First step in finding a solution is to get the previous end time next to start and end time in each record. We can use window functions in SQL to get this using something like below query.

In [ ]:
SELECT
name, 
floor_number,
start_time,
end_time,
ROW_NUMBER () OVER (ORDER BY name, start_time, end_time) rn,
max(end_time) over(partition by name, floor_number order by start_time, end_time ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) prev_end_time
from 
employee_work

Run the above query using duck db:

In [18]:
duckdb.sql('''SELECT
name, 
floor_number,
start_time,
end_time,
ROW_NUMBER () OVER (ORDER BY name, start_time, end_time) rn,
max(end_time) over(partition by name, floor_number order by start_time, end_time ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) prev_end_time
from 
employee_work''')

┌─────────┬──────────────┬─────────────────────┬─────────────────────┬───────┬─────────────────────┐
│  name   │ floor_number │     start_time      │      end_time       │  rn   │    prev_end_time    │
│ varchar │    int64     │       varchar       │       varchar       │ int64 │       varchar       │
├─────────┼──────────────┼─────────────────────┼─────────────────────┼───────┼─────────────────────┤
│ Mark    │            1 │ 2024-05-31 11:56:00 │ 2024-05-31 15:21:00 │     3 │ NULL                │
│ Mark    │            1 │ 2024-05-31 13:26:00 │ 2024-05-31 16:12:00 │     4 │ 2024-05-31 15:21:00 │
│ Mark    │            1 │ 2024-05-31 17:01:00 │ 2024-05-31 17:31:00 │     5 │ 2024-05-31 16:12:00 │
│ Ashok   │            1 │ 2024-05-31 10:32:00 │ 2024-05-31 11:28:00 │     1 │ NULL                │
│ Ashok   │            1 │ 2024-05-31 12:29:00 │ 2024-05-31 14:21:00 │     2 │ 2024-05-31 11:28:00 │
└─────────┴──────────────┴─────────────────────┴─────────────────────┴───────┴─────────────

The above query finds the previous end time per record for an employee based on all records above current record(UNBOUNDED PRECEDING) up until the record just above the current record (1 PRECEDING). Once we have the previous end time, we can now find if the duration(start and end time) of current records overlaps with previous record and if it overlaps, we can consider the record as part of same island otherwise mark the current record as new island. Marking island can be as well done using integer values with island0 as 0, island1 as 1. This in SQL can again be achieved using window function.

In [ ]:
with ovlap as
(SELECT
name, 
floor_number,
start_time,
end_time,
ROW_NUMBER () OVER (ORDER BY name, start_time, end_time) rn,
max(end_time) over(partition by name, floor_number order by start_time, end_time ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) prev_end_time
from 
employee_work) 
----CTE end---
SELECT 
name,
floor_number,
start_time,
end_time,
prev_end_time,
-- indicates in ordered output by displaying 1 for new island start
CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END is_new_island,
SUM(CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END) OVER(PARTITION BY name, floor order by rn) island_id
FROM
ovlap

From the above query we get the new island indicator, showing if the record is start of a new island as per the problem it would be new time duration for the employee, on same floor with non overlapping time. Once we have this we can get the min and max duration within an island to find the actual time duration worked by employee on a floor from across multiple overlapping records.

Running this using duckdb:

In [20]:
duckdb.sql('''with ovlap as
(SELECT
name, 
floor_number,
start_time,
end_time,
ROW_NUMBER () OVER (ORDER BY name, start_time, end_time) rn,
max(end_time) over(partition by name, floor_number order by start_time, end_time ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) prev_end_time
from 
employee_work) 
----CTE end---
SELECT 
name,
floor_number,
start_time,
end_time,
prev_end_time,
-- indicates in ordered output by displaying 1 for new island start
CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END is_new_island,
SUM(CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END) OVER(PARTITION BY name, floor_number order by rn) island_id
FROM
ovlap''')

┌─────────┬──────────────┬─────────────────────┬─────────────────────┬─────────────────────┬───────────────┬───────────┐
│  name   │ floor_number │     start_time      │      end_time       │    prev_end_time    │ is_new_island │ island_id │
│ varchar │    int64     │       varchar       │       varchar       │       varchar       │     int32     │  int128   │
├─────────┼──────────────┼─────────────────────┼─────────────────────┼─────────────────────┼───────────────┼───────────┤
│ Mark    │            1 │ 2024-05-31 11:56:00 │ 2024-05-31 15:21:00 │ NULL                │             1 │         1 │
│ Mark    │            1 │ 2024-05-31 13:26:00 │ 2024-05-31 16:12:00 │ 2024-05-31 15:21:00 │             0 │         1 │
│ Mark    │            1 │ 2024-05-31 17:01:00 │ 2024-05-31 17:31:00 │ 2024-05-31 16:12:00 │             1 │         2 │
│ Ashok   │            1 │ 2024-05-31 10:32:00 │ 2024-05-31 11:28:00 │ NULL                │             1 │         1 │
│ Ashok   │            1 │ 2024-

In [ ]:
with ovlap as
(SELECT
name, 
floor_number,
start_time,
end_time,
ROW_NUMBER () OVER (ORDER BY name, start_time, end_time) rn,
max(end_time) over(partition by name, floor_number order by start_time, end_time ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) prev_end_time
from 
employee_work) 
----CTE1 end---
islands as (
SELECT 
name,
floor_number,
start_time,
end_time,
prev_end_time,
-- indicates in ordered output by displaying 1 for new island start
CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END is_new_island,
SUM(CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END) OVER(PARTITION BY name, floor order by rn) island_id
FROM
ovlap)
---CTE end---
SELECT 
name, 
floor_number,
min(start_time) actual_start_time,
max(end_time) actual_end_time
FROM 
islands
group by 
name,
floor_number,
island_id


Finally this query gives the result with actual start and end time duration employee has worked on a floor with non overlapping durations as separate records. Subtracting actual end and start time on each row would give actual work duration.

In [24]:
duckdb.sql('''
with ovlap as
(SELECT
name, 
floor_number,
start_time,
end_time,
ROW_NUMBER () OVER (ORDER BY name, start_time, end_time) rn,
max(end_time) over(partition by name, floor_number order by start_time, end_time ROWS BETWEEN UNBOUNDED PRECEDING AND 1 PRECEDING) prev_end_time
from 
employee_work) ,
----CTE1 end---
islands as (
SELECT 
name,
floor_number,
start_time,
end_time,
prev_end_time,
-- indicates in ordered output by displaying 1 for new island start
CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END is_new_island,
SUM(CASE WHEN start_time <= prev_end_time then 0 ELSE 1 END) OVER(PARTITION BY name, floor_number order by rn) island_id
FROM
ovlap)
---CTE end---
SELECT 
name, 
floor_number,
min(start_time) actual_start_time,
max(end_time) actual_end_time
FROM 
islands
group by 
name,
floor_number,
island_id
order by name, floor_number, actual_start_time, actual_end_time
''')

┌─────────┬──────────────┬─────────────────────┬─────────────────────┐
│  name   │ floor_number │  actual_start_time  │   actual_end_time   │
│ varchar │    int64     │       varchar       │       varchar       │
├─────────┼──────────────┼─────────────────────┼─────────────────────┤
│ Ashok   │            1 │ 2024-05-31 10:32:00 │ 2024-05-31 11:28:00 │
│ Ashok   │            1 │ 2024-05-31 12:29:00 │ 2024-05-31 14:21:00 │
│ Mark    │            1 │ 2024-05-31 11:56:00 │ 2024-05-31 16:12:00 │
│ Mark    │            1 │ 2024-05-31 17:01:00 │ 2024-05-31 17:31:00 │
└─────────┴──────────────┴─────────────────────┴─────────────────────┘